# Modularity
Implements formula's from Lambiotte et al., 2008.

In [1]:
import numpy as np

In [33]:
# Create adjacency matrix
n = 4
A = np.zeros((n, n))
A[0,1] = 1
A[1,2] = 1
A[2,3] = 1
A[0,3] = 1
A[0,2] = 1
A = A + np.triu(A).T
m = A.sum() / 2
A

array([[ 0.,  1.,  1.,  1.],
       [ 1.,  0.,  1.,  0.],
       [ 1.,  1.,  0.,  1.],
       [ 1.,  0.,  1.,  0.]])

In [18]:
# assign to commnities
ncom = 2
C = np.zeros((n, ncom))
C[0, 0] = 1
C[1, 0] = 1
C[2, 1] = 1
C[3, 1] = 1
C

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [52]:
def get_modularity_matrix(mat):
    # k_i is the degree of k
    k = mat.sum(axis=0)
    mod = mat - (np.outer(k, k) / np.sum(mat))
    return mod

In [46]:
def get_modularity_value(mat, partition):
    # delta tells if i,j are in the same cluster
    delta = partition.dot(partition.T)
    mod = get_modularity_matrix(mat)
    Q = 1.0 / np.sum(mat) * np.sum(delta*mod)
    return Q

In [63]:
def get_init_stability(mat, partition):
    k = mat.sum(axis=0)
    delta = partition.dot(partition.T)
    return 1 - np.sum(delta*np.outer(k, k)/ np.square(sum(mat)))

In [65]:
def get_approx_stability(mat, partition, t):
    stab_init = get_init_stability(mat, partition)
    modularity_value = get_modularity_value(mat, partition)
    return (1.0-t)*stab_init + t*modularity_value

In [56]:
Q = get_modularity_value(A, C)
Q

-0.10000000000000001

In [57]:
B = get_modularity_matrix(A)
B

array([[-0.9,  0.4,  0.1,  0.4],
       [ 0.4, -0.4,  0.4, -0.4],
       [ 0.1,  0.4, -0.9,  0.4],
       [ 0.4, -0.4,  0.4, -0.4]])

In [64]:
get_init_stability(A, C)

-7.3333333333333321

In [66]:
get_approx_stability(A, C, 0.5)

-3.7166666666666659

## compare with reference implementation

In [67]:
import networkx as nx
import community

In [95]:
graph = nx.Graph(A)
nz = C.nonzero()
partition = {nz[0][i]: nz[1][i] for i in range(len(nz[0]))}
community.modularity(partition, graph)

-0.09999999999999998